<a href="https://colab.research.google.com/github/RaminVali/CL_WebScraping_ML_Prediction/blob/main/scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setting up Selenium Chorme dirver to use for scraping and scrolling donwn etc.

Google colab has issues with selenium and the associated websidrver because their ubunto release does not include snap. Here is a workaround for this issue on google colab. Please see [here](https://stackoverflow.com/questions/75164313/selenium-in-google-colab-stopped-working-showing-an-error-as-service-chromedrive) for more details.

In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get install chromium chromium-driver
apt-get update


# Install selenium
pip install selenium

# Install
pip install webdriver-manager

In [ ]:
! ls /usr/bin/chromedriver

/usr/bin/chromedriver


Importing Packages

In [88]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException
import os
from bs4 import BeautifulSoup
from urllib.parse import urljoin


import pandas as pd

## Setting up the webdriver with appropriate options
To not be flagged as a robot by the website, see [here](https://stackoverflow.com/questions/71885891/urllib3-exceptions-maxretryerror-httpconnectionpoolhost-localhost-port-5958)

In [20]:


#service = Service(executable_path='/usr/lib/chromium-browser/chromedriver/usr/bin') #this gives a version mismatch error bweteen the driver and the Chormium

def get_driver():
  service = Service(executable_path='/usr/bin/chromedriver') #this is where the correct version of the chromium and webdriver live


  options = Options()
  options.add_argument("--headless")
  options.add_argument("--no-sandbox")
  options.add_argument('--disable-dev-shm-usage')
  options.add_argument('--disable-blink-features=AutomationControlled')

  driver = webdriver.Chrome(service = service, options = options)
  return driver


Setting up the driver and doing infinite scrolling to get all the html form the main website page. We will get the hrefs with all the links. [Source](https://medium.com/analytics-vidhya/using-python-and-selenium-to-scrape-infinite-scroll-web-pages-825d12c24ec7)

In [ ]:
# Web scrapper for infinite scrolling page
driver = get_driver()
driver.get("https://www.privateislandsonline.com/search?availability=sale&region=&q2=&price_range=0%3A50000000&size_range=0%3A1000")
time.sleep(2)  # Allow 2 seconds for the web page to open
scroll_pause_time = 2 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
i = 1

while True:
    # scroll one screen height each time
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))
    i += 1
    time.sleep(scroll_pause_time)
    # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
    scroll_height = driver.execute_script("return document.body.scrollHeight;")
    # Break the loop when the height we need to scroll to is larger than the total scroll height
    if (screen_height) * i > scroll_height:
        break



soup = BeautifulSoup(driver.page_source, "html.parser")


Saving the contents we scraped

In [ ]:
with open("output1.html", "w", encoding='utf-8') as file:
    file.write(str(soup))

Loading the saved contents again

In [21]:
path = '/content/drive/MyDrive/Colab Notebooks/Private_Island_Project/Island_Project/output1.html'

with open(path, 'r') as f:
    contents = f.read()
    soup = BeautifulSoup(contents, 'lxml')

##### Extract Individual Island URLs #####
urls = []
for parent in soup.find_all('div', class_="img-container"):
    url = parent.find('a').get('href')
    urls.append(url)

print(f"We have found {len(urls)} islands for sale")

We have found 611 islands for sale


We will go through each 611 URLs and dump the page contents into a seperate folder. The folder name is going to be from the URL, where it is region/country/island_name or region/region/country/island_name.

In [22]:
test_url = urls[0:2]
test_url

['https://www.privateislandsonline.com/asia/thailand/rangyai-island',
 'https://www.privateislandsonline.com/europe/spain/2867']

In [69]:
DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/Private_Island_Project/Island_Project/Data"


In [164]:
def get_html(url, sleep = 5, retries = 3):
  #time.sleep(sleep)
  soup = None
  for i in range (1, retries+1):
    Driver = get_driver()

    try:
      time.sleep(sleep)  # Allow 2 seconds for the web page to open
      scroll_pause_time = 2 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
      screen_height = Driver.execute_script("return window.screen.height;")   # get the screen height of the web
      i = 1

      while True:
          # scroll one screen height each time
          Driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))
          i += 1
          time.sleep(scroll_pause_time)
          # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
          scroll_height = Driver.execute_script("return document.body.scrollHeight;")
          # Break the loop when the height we need to scroll to is larger than the total scroll height
          if (screen_height) * i > scroll_height:
              break

      soup = BeautifulSoup(Driver.page_source, "html.parser")

    except TimeoutException as ex:
        isrunning = 0
        print(f"Timeout error on {url} the exception is: {ex}")
        Driver.close()
        continue
    else:
        break
  return soup

def make_path(url):
  # make file name
  part = url.split("/")
  if len(part) == 6:
    file_name = part[-3]+'-'+part[-2]+'-'+part[-1]
  else:
    file_name = part[-4]+'-'+part[-3]+'-'+part[-2]+'-'+part[-1]

  save_path = os.path.join(DATA_DIR,file_name)
  return save_path, file_name

def save_html(all_html, save_path):
  with open(save_path+".mhtml", "w+", encoding='utf-8') as file:
    file.write(str(all_html))


In [165]:
for url in test_url:

  (save_path, file_name) = make_path(url)
  if os.path.exists(save_path):
    continue

  get_html(url)

  save_html(soup, save_path)




In [79]:
islands_files = os.listdir(DATA_DIR)
islands_files = [i for i in islands_files if '.html' in i]

In [132]:
islands_files

['europe-spain-2867.html', 'asia-thailand-rangyai-island.html']

In [148]:
with open(islands_files[0], 'r') as f:
  html = f.read()

soup = BeautifulSoup(html)
basic_info = soup.select("div", {'class':'class="basic-info mb-50"'})
basic_info

[<div class="navOverlay"></div>,
 <div class="container-fluid clearfix"><div class="navbar-header"><a class="navbar-brand" href="https://www.privateislandsonline.com"><img alt="new-logo" class="img-responsive desktopBigLogo" src="https://www.privateislandsonline.com/img/front/new-logo.png"/><img alt="logo small" class="mobileSmallLogo" src="https://www.privateislandsonline.com/img/front/logo_small.png"/></a></div><div id="side-menu"><button class="" type="button"><svg height="512px" id="Capa_1" style="enable-background:new 0 0 396.667 396.667;" version="1.1" viewbox="0 0 396.667 396.667" width="512px" x="0px" xml:space="preserve" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" y="0px"><g><g><path d="M17,87.833h362.667c9.35,0,17-7.65,17-17s-7.65-17-17-17H17c-9.35,0-17,7.65-17,17C0,80.183,7.65,87.833,17,87.833z" fill="#FFFFFF"></path><path d="M17,215.333h362.667c9.35,0,17-7.65,17-17s-7.65-17-17-17H17c-9.35,0-17,7.65-17,17S7.65,215.333,17,215.333z" fill="#FFF

In [130]:
print(basic_info)

None


In [141]:
content = soup.find('basic-info mb-50')

print(content)

None


In [104]:
df = pd.read_html(str(soup), attrs = {'class': 'basic-info mb-50'})[0]

ValueError: ignored

In [ ]:
basic_info

In [86]:
id = 'island-details'

In [ ]:
html